In [36]:
def hello(thing):
    return "Hello " + thing


In [37]:
hello("python WA")


'Hello python WA'

In [38]:
hello(3)


TypeError: can only concatenate str (not "int") to str

In [39]:
def hello(thing: str) -> str:
    return "Hello " + thing


In [40]:
hello(33)


TypeError: can only concatenate str (not "int") to str

In [41]:
import datetime 


def day_of_week(lookup_date: datetime.date) -> str:
    return lookup_date.weekday()


In [42]:

day_of_week("2000-01-01")


AttributeError: 'str' object has no attribute 'weekday'

In [43]:
day_of_week(datetime.date(2000,1,1))


5

In [44]:
def day_of_week(lookup_date: datetime.date) -> str:
    return lookup_date.strftime('%A')


In [45]:
day_of_week(datetime.date(2000,1,1))

'Saturday'

## classes

In [46]:

class Person:
    def __init__(self, name: str, birthday: datetime.date, gender: str) -> None:
        self.name : str = name
        self.birthday: datetime.date = birthday
        self.gender: str = gender
        
    def day_of_birth(self) -> str:
        return day_of_week(self.birthday)
    
    def __repr__(self) -> str:
        return f"{self.name} ({self.birthday}) {self.gender}"

In [47]:
jane = Person("Jane", datetime.date(2000,1,1))

TypeError: Person.__init__() missing 1 required positional argument: 'gender'

In [48]:
jane: Person = Person("Jane", datetime.date(2000,1,1), "female")
jane


Jane (2000-01-01) female

In [49]:
class Person:
    def __init__(self, name: str, birthday: datetime.date, gender: str | None = None) -> None:
        self.name : str = name
        self.birthday: datetime.date = birthday
        self.gender: str | None = gender
        
    def day_of_birth(self) -> str:
        return day_of_week(self.birthday)
    
    def __repr__(self) -> str:
        return f"{self.name} ({self.birthday}) {self.gender}"

In [66]:
jane: Person = Person("Jane", datetime.date(2000,1,1))
jane

Jane (2000-01-01) None

In [68]:
jane.day_of_birth().capitalize()

'Saturday'

In [69]:
jane.day_of_birth() / 2

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [71]:
# type hints are not enforced
jane.gender = 1

In [72]:
jane

Jane (2000-01-01) 1

In [75]:
jane.gender.split()

AttributeError: 'int' object has no attribute 'split'

In [76]:
jane.gender / 2

0.5

## Cool tools

### typing.NamedTuple

In [51]:
import typing


class Employee(typing.NamedTuple):
    first_name: str
    last_name: str
    country: str
    jobs: list[str]
    
james = Employee("james", "bond", "United Kingdom", ["jet setter", "spy", "secret agent"])

In [52]:
james

Employee(first_name='james', last_name='bond', country='United Kingdom', jobs=['jet setter', 'spy', 'secret agent'])

In [53]:
james.first_name

'james'

In [54]:
james.jobs.index("spy")

1

### typing.TypedDict

In [55]:
class Point2D(typing.TypedDict):
    x: int
    y: int
    label: str

In [56]:
a: Point2D = {'x': 1, 'y': 2, 'label': 'good'}  # OK

In [57]:
b: Point2D = {'x': 1, 'label': 'bad'}           # Fails type check

In [58]:
b

{'x': 1, 'label': 'bad'}

In [59]:
class Point2D(typing.TypedDict):
    x: int
    y: int
    label: typing.NotRequired[str]

In [60]:
b: Point2D = {'x': 1, 'y': 2}           # Fails type check

In [61]:
# adding an extra value 
c: Point2D = {'x': 1, 'y': 2, 'z': 3}

In [62]:
class Point2D(typing.TypedDict, total=False):
    x: int
    y: int

In [63]:
b: Point2D = {'x': 1, 'y': 2}    

In [64]:
c: Point2D = {'x': 1, 'y': 2, 'z': 3}

In [65]:
# a typed dict is the equivalent of a dict 
Point2D(x=1, y=2, label='first') == dict(x=1, y=2, label='first')

True

### data classes


In [92]:
import dataclasses


@dataclasses.dataclass
class InventoryItem:
    """Class for keeping track of an item in inventory."""
    name: str
    unit_price: float
    quantity_on_hand: int = 0

    def total_cost(self) -> float:
        return self.unit_price * self.quantity_on_hand


In [93]:
item = InventoryItem("test item", 20.00, 6)
dataclasses.asdict(item) == {"name": "test item", "unit_price": 20.00, "quantity_on_hand": 6}

True

In [94]:
item.name

'test item'

In [95]:
item.total_cost()

120.0

In [96]:
# i wonder if this will raise a warning in VS Code? 
item.quantity_on_hand = "16"

In [106]:
item.total_cost()


160.0

#### post_init

In [107]:
@dataclasses.dataclass
class InventoryItem:
    """Class for keeping track of an item in inventory."""
    name: str
    unit_price: float
    quantity_on_hand: int = 0
    date_created: datetime.datetime = dataclasses.field(init=False)
    
    def __post_init__(self):
        self.date_created = datetime.datetime.now()

    def total_cost(self) -> float:
        return self.unit_price * self.quantity_on_hand


In [108]:
item = InventoryItem("test item", 20.00, 6)
item

InventoryItem(name='test item', unit_price=20.0, quantity_on_hand=6, date_created=datetime.datetime(2023, 6, 1, 13, 7, 26, 175093))

#### default factory

In [115]:
@dataclasses.dataclass
class InventoryItem:
    """Class for keeping track of an item in inventory."""
    name: str
    unit_price: float
    quantity_on_hand: int = 0
    date_created: datetime.datetime = dataclasses.field(init=False)
    stock_history: list[int] = dataclasses.field(default_factory=list)
    
    def __post_init__(self):
        self.date_created = datetime.datetime.now()
        
    def __add__(self, other:int):
        """ override the + method """
        if type(other) != int:
            raise ValueError("not an int")
        self.quantity_on_hand += other
        self.stock_history.append(other)
        
    def __sub__(self, other):
        """ override the - method """
        self.__add__(other * -1)

    def total_cost(self) -> float:
        return self.unit_price * self.quantity_on_hand

In [116]:
item = InventoryItem("test item", 20.00, 6)
item

InventoryItem(name='test item', unit_price=20.0, quantity_on_hand=6, date_created=datetime.datetime(2023, 6, 1, 13, 9, 17, 972598), stock_history=[])

In [117]:
item + 1
item

InventoryItem(name='test item', unit_price=20.0, quantity_on_hand=7, date_created=datetime.datetime(2023, 6, 1, 13, 9, 17, 972598), stock_history=[1])

In [118]:
item - 1
item


InventoryItem(name='test item', unit_price=20.0, quantity_on_hand=6, date_created=datetime.datetime(2023, 6, 1, 13, 9, 17, 972598), stock_history=[1, -1])

In [119]:
item + 10
item - 9
item + 12
item - 6
item


InventoryItem(name='test item', unit_price=20.0, quantity_on_hand=13, date_created=datetime.datetime(2023, 6, 1, 13, 9, 17, 972598), stock_history=[1, -1, 10, -9, 12, -6])